In [1]:
import pandas as pd
import os
from PIL import Image
import pandas as pd
from tqdm.notebook import trange, tqd

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
import base64
from io import BytesIO

#Convert Image to Bytes
def img_2_bytes(image):
    buff = BytesIO()
    image.save(buff, format="JPEG")
#    img_str = base64.b64encode(buff.getvalue())
    img_str = buff.getvalue()

    return img_str

# Convert Bytes to Image
def bytes_2_img(data):
#    buff = BytesIO(base64.b64decode(data))
    buff = BytesIO(data)

    return Image.open(buff

In [ ]:
def load_images_from_directory(src):
    TARGET_SIZE = (223, 223)

    data = dict()
    data['labels'] = []
    data['images'] = []
    data['resized'] = []
    data['filename'] = []

    for root, subdir, files in os.walk(src):
        for class_name in subdir:
            class_dir = os.path.join(root, class_name)
            for filename in os.listdir(class_dir):
                img_path = os.path.join(class_dir, filename)
                with Image.open(img_path) as img:
                    resized_img = img
                    data['labels'].append(class_name)
                    data['resized'].append(img_2_bytes(resized_img.resize(TARGET_SIZE, Image.Resampling.LANCZOS)))
                    data['images'].append(img_2_bytes(img))
                    data['filename'].append(filename)
    return data

In [ ]:
directory   = fr'.\FAITH-Bark\Tree'
dataset = load_images_from_directory(directory)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def visualize(image, transform=None, x=200, y=200):
    plt.figure(figsize=(x, y))
    plt.axis('off')
    plt.imshow(image)

In [ ]:
image_df = pd.DataFrame.from_dict(dataset, orient='index').transpose()
os.makedirs(r'.\out', exist_ok=True)
image_df.to_parquet(path=r'.\out/dataset.parquet')

In [ ]:
from time import sleep
def progress(ln):
    for i in tqdm(range(0, ln), bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}]"):
        sleep(0.5

Extracting the image from dataframe explicitly from the known column name

In [ ]:
def extract_df_img(dataset, path):
    path = path
    subfolder_names = dataset['labels'].unique().tolist()
    ds_len = len(dataset)
    progress(ds_len)
    for i in range(ds_len):
        decoded = bytes_2_img(dataset.loc[i, 'resized'])
        # Access the subfolder names
        if dataset.loc[i, 'labels'] in subfolder_names:
            subfolder_name = dataset.loc[i, 'labels']
            subfolder_path = os.path.join(path, subfolder_name)
            os.makedirs(subfolder_path, exist_ok=True) # Create the subfolder from label
            # Save the folder
            decoded.save(os.path.join(subfolder_path, dataset.loc[i, 'filename']), format='JPEG')
    
    print("Success")

In [ ]:
PATH = '/out/Tree'
extract_df_img(dataset, PATH)

In [ ]:
import shutil

# Specify the path to the folder you want to zip
folder_to_zip = '/content/out'

# Specify the name for the zip file
zip_filename = 'FAITHBark'

# Create the zip file
shutil.make_archive(zip_filename, 'zip', folder_to_zip)